# Dataset creation with [Polaris](https://github.com/polaris-hub/polaris)

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pathlib

import pandas as pd
import datamol as dm

# polaris dataset
from polaris.dataset import Dataset, ColumnAnnotation
from polaris.utils.types import HubOwner


root = pathlib.Path("__file__").absolute().parents[1]
os.chdir(root)
sys.path.insert(0, str(root))

In [2]:
# Get the owner and organization
org = "AdaptyvBio"
data_name = "IL7Ra_binders"
dirname = dm.fs.join(root, f"org-{org}",data_name)
gcp_root = f"gs://polaris-public/polaris-recipes/org-{org}/{data_name}"

owner = HubOwner(slug="adaptyv-bio", type="organization")
owner

HubOwner(slug='adaptyv-bio', external_id=None, type='organization')

In [4]:
BENCHMARK_DIR = f"gs://polaris-public/polaris-recipes/org-{org}/{data_name}/benchmarks"
DATASET_DIR = f"gs://polaris-public/polaris-recipes/org-{org}/{data_name}/datasets"
FIGURE_DIR = f"gs://polaris-public/polaris-recipes/org-{org}/{data_name}/figures"

## Load existing data

In [5]:
# Load the curated data
PATH = "gs://polaris-public/polaris-recipes/org-AdaptyvBio/raw/IL7Ra_binders_data.csv"
table = pd.read_csv(PATH)

### Below we specify the meta information of data columns

It's necessary to specify the key bioactivity columns, molecule structures and identifiers in the dataset with `ColumnAnnotation`. It is possible to add `user_attributes` with any key and values when needed, such as `unit`, `organism`, `scale` and optimization `objective`. 

In [15]:
# Here we simplify the column names
table = table.rename(
    columns={
        "KD (M)": "KD"
    }
)
# fill nans
table["Binder_class"] = table["Binder_class"].fillna("inactive").replace("none", "inactive").values
table["Binder_class"].value_counts()

Binder_class
inactive    61
strong      19
weak        15
Name: count, dtype: int64

In [18]:
annotations = {
    "sequence": ColumnAnnotation(
        description="Protein sequence in fasta format"
    ),
    "KD": ColumnAnnotation(
        description="Kd for the measure of binding affinity.",
            user_attributes={
            "unit": "M", 
            "objective": "Lower value",}
    ),
    "Binder_class": ColumnAnnotation(
        description="The binding affinity as three classes labels.",
        user_attributes={
            "objective": "strong",}
    ),
    "Binder_bool": ColumnAnnotation(
        description="The binding affinity as boolean class label",
        user_attributes={
            "objective": "True",
        },
    ),
}

### Define `Dataset` object

In [17]:
dataset_version = "v1"
dataset_name = f"IL7Ra_binders-{dataset_version}"

In [20]:
dataset = Dataset(
    table=table[annotations.keys()].copy(),
    name=dataset_name,
    description="This dataset includes binding protein designs targeting the interleukin-7 receptor alpha chain (IL7RA), a drug target associated with various diseases.",
    source="https://www.adaptyvbio.com",
    annotations=annotations,
    owner=owner,
    tags=["protein-design"],
    license="CC-BY-4.0")

### Dataset overview

In [24]:
dataset

{
  "name": "IL7Ra_binders-v1",
  "description": "This dataset includes binding protein designs targeting the interleukin-7 receptor alpha chain (IL7RA), a drug target associated with various diseases.",
  "tags": [
    "protein-design"
  ],
  "user_attributes": {},
  "owner": "adaptyv-bio",
  "polaris_version": "0.7.9",
  "default_adapters": {},
  "zarr_root_path": null,
  "readme": "",
  "annotations": {
    "sequence": {
      "is_pointer": false,
      "modality": "UNKNOWN",
      "description": "Protein sequence in fasta format",
      "user_attributes": {},
      "dtype": "object"
    },
    "KD": {
      "is_pointer": false,
      "modality": "UNKNOWN",
      "description": "Kd for the measure of binding affinity.",
      "user_attributes": {
        "unit": "M",
        "objective": "Lower value"
      },
      "dtype": "object"
    },
    "Binder_class": {
      "is_pointer": false,
      "modality": "UNKNOWN",
      "description": "The binding affinity as three classes labels.",
      "user_attributes": {
        "objective": "strong"
      },
      "dtype": "object"
    },
    "Binder_bool": {
      "is_pointer": false,
      "modality": "UNKNOWN",
      "description": "The binding affinity as boolean class label",
      "user_attributes": {
        "objective": "True"
      },
      "dtype": "bool"
    }
  },
  "source": "https://www.adaptyvbio.com",
  "license": "CC-BY-4.0",
  "curation_reference": null,
  "cache_dir": "/Users/lu.zhu/Library/Caches/polaris/datasets/69ed8871-f69b-453b-adef-e34e76db9898",
  "md5sum": "7b6821da4069ec6a96603ac3cc238d3b",
  "artifact_id": "adaptyv-bio/il7ra-binders-v1",
  "n_rows": 95,
  "n_columns": 4
}

In [21]:
# save the dataset to GCP
SAVE_DIR = f"{DATASET_DIR}/{dataset_name}"
dataset_path = dataset.to_json(SAVE_DIR)
dataset_path

'gs://polaris-public/polaris-recipes/org-AdaptyvBio/IL7Ra_binders/datasets/IL7Ra_binders-v1/dataset.json'

### Upload the dataset to the hub

In [23]:
dataset.upload_to_hub()

💥 ERROR: Failed to upload dataset. 


/Users/lu.zhu/miniconda3/envs/po_dataset/lib/python3.12/site-packages/yaspin/core.py:228: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(value) if value else value


PolarisCreateArtifactError: The request to the Polaris Hub failed. The Hub responded with:
{
  "message": "Cannot create a dataset for owner 'adaptyv-bio'"
}
[1m[93mNote: If you can confirm that you are authorized to perform this action, please call 'polaris login --overwrite' and try again. If the issue persists, please reach out to the Polaris team for support.[0m